In [1]:
import sys
sys.path.append('../')

import meta_dataloader.TCGA

import models.mlp, models.gcn
import numpy as np
import data.gene_graphs
import collections
import sklearn.metrics
import sklearn.model_selection
import pandas as pd
%load_ext autoreload
%autoreload 2

import torch

In [2]:
path = "../data/MBdata_original.csv"
df = pd.read_csv(path)

df = df[df.Pam50Subtype != '?']
target = df.pop('Pam50Subtype')
pam50_lables = {
    'Normal': 0,
    'LumA': 1,
    'LumB': 2,
    'Basal': 3,
    'Her2': 4
}
target = target.apply(lambda x: pam50_lables[x])

features = df.filter(regex='^GE.*')
features = features.astype('float64')

/auto/homes/mt675/gene-graph-conv-onto/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def normalize(df):
    r = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if max_value != min_value:
          r[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return r

In [4]:
features = df.filter(regex='^GE.*')
features = normalize(features)
features.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
features = features.rename(columns={g: g[3:] for g in list(features.columns)})

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [5]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(features, 
                                                                        target.to_numpy(), 
                                                                        stratify=target.to_numpy(),
                                                                        train_size=100,
                                                                        test_size=400,
                                                                        shuffle=True,
                                                                        random_state=0
                                                                        )




In [86]:
from data.gene_graphs import GeneManiaGraph, OntologyGraph, StringDBGraph
gene_graph =  OntologyGraph(neighbors=101, embeddings_name='dl2vec')

 loaded genes. nodes=17591


17591it [00:35, 495.32it/s]


 loaded edges=1558172
 writing graph


In [87]:
gene = 'ESR1'
num_genes = 20000
neighbors = gene_graph.bfs_sample_neighbors(gene, num_genes*1.5)
intersection_nodes = np.intersect1d(X_train.columns, neighbors.nodes)

x_train = X_train[list(intersection_nodes)[:num_genes]].copy()
x_test = X_test[list(intersection_nodes)[:num_genes]].copy()

toremove = set(neighbors.nodes)
toremove = toremove.difference(intersection_nodes)
neighbors.remove_nodes_from(toremove)

In [88]:
x_train

,A1BG,A1CF,A2M,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
1294,0.636301,0.147265,0.134259,0.297461,0.512303,0.393817,0.247217,0.112401,0.079587,0.121226,...,0.132780,0.388961,0.572912,0.340735,0.493843,0.189562,0.407518,0.603684,0.739919,0.260905
837,0.458253,0.138490,0.469556,0.595413,0.275540,0.202664,0.232734,0.215425,0.121300,0.150613,...,0.151944,0.208716,0.597135,0.324473,0.597460,0.137628,0.463331,0.664662,0.934488,0.406994
1621,0.501620,0.162249,0.331539,0.517617,0.224598,0.352779,0.269712,0.389618,0.133153,0.120376,...,0.138593,0.179974,0.381344,0.051494,0.582036,0.098780,0.392550,0.707985,0.542029,0.273688
705,0.320893,0.113174,0.553758,0.567101,0.168377,0.385363,0.306095,0.229193,0.162149,0.186637,...,0.196402,0.125435,0.382977,0.220131,0.327394,0.164213,0.347799,0.582531,0.626735,0.347536
783,0.520771,0.101000,0.449263,0.549906,0.355561,0.367882,0.436045,0.347727,0.113716,0.139733,...,0.267811,0.350238,0.518407,0.203420,0.335965,0.149116,0.467493,0.485741,0.721266,0.397768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,0.374074,0.055278,0.432433,0.416246,0.249548,0.330350,0.236124,0.163306,0.111365,0.164631,...,0.259699,0.493719,0.454614,0.268301,0.445756,0.210657,0.293436,0.522969,0.492630,0.265062
820,0.408940,0.137501,0.502431,0.641476,0.357350,0.381450,0.306619,0.201469,0.146445,0.138753,...,0.225571,0.349179,0.416444,0.110326,0.369813,0.137045,0.279570,0.623411,0.614479,0.343611
1452,0.492988,0.138968,0.612952,0.600142,0.469619,0.330885,0.298594,0.230955,0.122827,0.128361,...,0.219100,0.169508,0.409179,0.325230,0.379185,0.180482,0.534211,0.633205,0.613331,0.301850
558,0.496006,0.084515,0.509258,0.526464,0.572258,0.353164,0.744023,0.176307,0.133995,0.116350,...,0.152378,0.294727,0.449277,0.116760,0.581970,0.065802,0.762843,0.399978,0.736339,0.580970


In [78]:
from scipy import sparse
import networkx as nx

adj = sparse.csr_matrix(nx.to_numpy_matrix(neighbors))


In [96]:
from models.gcn import GCN
model = GCN(name="GCN_noemb2_lay1_chan64_dropout_agg_hierarchy", dropout=False, cuda=True, num_layer=1, prepool_extralayers=1, channels=64, embedding=64, aggregation="random", lr=0.001, num_epochs=100, patience=20, verbose=False, seed=2)


In [97]:
model.fit(x_train, y_train, adj=adj)


/auto/homes/mt675/gene-graph-conv-onto/venv/lib/python3.6/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Reducing graph by a factor of 2 to 7154 nodes


RuntimeError: index_add_(): Number of indices should be equal to self.size(dim)

In [ ]:
y_hat = model.predict(x_test)
y_hat = np.argmax(y_hat, axis=1)
# auc = sklearn.metrics.roc_auc_score(y_test, np.asarray(y_hat).flatten(), multi_class='ovo')
acc = sklearn.metrics.accuracy_score(y_test, np.asarray(y_hat).flatten())
acc

In [13]:
model.get_representation()

AttributeError: 'GCNLayer' object has no attribute 'in_features'

In [14]:
model.conv_layers

ModuleList(
  (0): GCNLayer(
    (linear): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
    (eye_linear): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
  )
)

In [15]:
model.my_logistic_layers.parameters()

<generator object Module.parameters at 0x7efff8385d00>

In [16]:
model.centroids

[array([  0, 119, 521, 698, 585, 467, 449, 697, 926, 686, 509, 743, 280,
        257, 544, 401, 708, 989, 192, 640, 318, 634,   9, 693, 761, 890,
        811, 854, 587, 522, 532, 101, 598, 543, 610, 648, 679, 227, 779,
        978, 376, 841, 739, 128, 114,  50, 150, 791,  55, 659, 891, 154,
        354, 267, 263, 179, 867, 514, 984, 674, 975, 433, 863, 703, 754,
        256, 582, 988, 453, 269, 879, 356, 726, 483, 147, 480, 351, 232,
        133, 288, 577, 466, 731, 465, 235, 728, 727, 616,  81, 203, 954,
        881, 388, 472, 562, 559, 614, 667, 900, 152, 441, 438, 405, 973,
        487, 151, 813, 662, 121, 664,  93, 613,  14, 187,  80, 271, 802,
        785,  73, 591, 555, 248, 247, 329, 736, 737, 623, 528, 503, 547,
        108, 113, 556, 747, 511, 563, 139, 692, 334, 990, 258, 129,  17,
        212,  36, 149, 276, 284, 684, 714, 324, 695, 745, 654, 827, 500,
        956, 196, 606, 797, 407, 328, 800,  21, 378, 525,  48, 109, 930,
        358, 539,  76, 600, 451, 592, 557, 985,  70

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [22]:
64*64

4096

In [46]:
count_parameters(model)

389125

In [10]:
from models.mlp import MLP
model2 = MLP(name="MLP_lay2_chan64", cuda=True, dropout=False, num_layer=2, channels=64, lr=0.001, patience=30)
model2.fit(X_train, y_train)


INFO:root:Constructing the network...
INFO:root:Done!


In [11]:
model2.eval()
y_hat = model2.predict(X_test)
y_hat = np.argmax(y_hat, axis=1)
# auc = sklearn.metrics.roc_auc_score(y_test, np.asarray(y_hat).flatten(), multi_class='ovo')
acc = sklearn.metrics.accuracy_score(y_test, np.asarray(y_hat).flatten())
acc

0.6375

In [70]:
count_parameters(model2)


1564101

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
for lr in [0.0001,0.001]:
    for seed in [0,1]:
        for num_layer in [1,2,3]:
            model = models.mlp.MLP(name="MLP_lay1_chan256",
                                   num_layer=num_layer, 
                                   channels=64, 
                                   lr=lr,
                                   patience=30,
                                   cuda=True,
                                   metric=sklearn.metrics.accuracy_score,
                                   verbose=False,
                                   seed=seed)

            model.fit(X_train, y_train)

            y_valid_pred = model.predict(X_test)
            print(seed, lr, num_layer, sklearn.metrics.accuracy_score(y_test, np.argmax(y_valid_pred,axis=1)))

INFO:root:Constructing the network...
INFO:root:Done!
INFO:root:Constructing the network...
INFO:root:Done!


0 0.0001 1 0.75


INFO:root:Constructing the network...
INFO:root:Done!


0 0.0001 2 0.775


INFO:root:Constructing the network...
INFO:root:Done!


...

0 0.001 3 0.7475


INFO:root:Constructing the network...
INFO:root:Done!


1 0.001 1 0.765


INFO:root:Constructing the network...
INFO:root:Done!


1 0.001 2 0.755
1 0.001 3 0.7725


In [17]:
acc = []

for seed in range(5):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(features, 
                                                        target.to_numpy(), 
                                                        stratify=target.to_numpy(),
                                                        train_size=1000,
                                                        test_size=400,
                                                        shuffle=True,
                                                        random_state=seed
                                                        )
    model2 = MLP(name="MLP_lay2_chan64", cuda=True, dropout=False, num_layer=2, channels=64, lr=0.0001, patience=30, seed=seed)
    model2.fit(X_train, y_train)
    model2.eval()
    y_hat = model2.predict(X_test)
    y_hat = np.argmax(y_hat, axis=1)
    a = sklearn.metrics.accuracy_score(y_test, np.asarray(y_hat).flatten())
    print(a)
    acc.append(a)

print(np.mean(acc), np.std(acc))

INFO:root:Constructing the network...
INFO:root:Done!
INFO:root:Constructing the network...
INFO:root:Done!
INFO:root:Constructing the network...
INFO:root:Done!
INFO:root:Constructing the network...
INFO:root:Done!
INFO:root:Constructing the network...
INFO:root:Done!


0.7315 0.023484037131634748


In [49]:
for n in [500, 100, 10]:
    for emb in ['el', 'dl2vec', 'opa2vec', 'opa2vec_go']:
        g = data.gene_graphs.OntologyGraph(neighbors=n, embeddings_name=emb) 
        print(n,emb,len(g.nx_graph.nodes), len(g.nx_graph.edges))
        
        

 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/ontology_el_500.adjlist.gz
500 el 13866 4600773
 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/ontology_dl2vec_500.adjlist.gz
500 dl2vec 17591 7976452
 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/ontology_opa2vec_500.adjlist.gz
500 opa2vec 20258 8401561
 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/ontology_opa2vec_go_500.adjlist.gz
500 opa2vec_go 17580 7320451
 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/ontology_el_100.adjlist.gz
100 el 13866 979948
 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/ontology_dl2vec_100.adjlist.gz
100 dl2vec 17591 1542359
 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/ontology_opa2vec_100.adjlist.gz
100 opa2vec 20258 1731962
 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/ontology_opa2

In [21]:
for g in [data.gene_graphs.GeneManiaGraph(), 
            data.gene_graphs.StringDBGraph(datastore="../data")]:
    print(len(g.nx_graph.nodes), len(g.nx_graph.edges))

 loading from cache file/auto/homes/mt675/gene-graph-conv-onto/data/graphs/genemania.adjlist.gz
 loading from cache file../data/graphs/stringdb_graph_all_edges.adjlist
16300 264657
18851 5703510
